In [92]:
%pip install scikit-learn
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: scikit-learn in c:\users\ofeks\onedrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\lib\site-packages (1.2.1)



You should consider upgrading via the 'c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [93]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
import pandas as pd

##### Train the model from the Bonus Assignment

In [95]:
import xgboost as xgb

def train_bonus_ass_model():
    """
        The function repeats the training process of the model in the
        Bonus assignment. Returned values are the trained model and the selected values.

        Features selection is done by '''SelectPercentile'''.
        Model used is XGBoost.
        Test is 20% of total data.
    """
    df = pd.read_csv("../BonusAssignment/text_training.csv", usecols=list(range(1,2002)))

    # separate the features and target variable
    features = df.iloc[:, 1:-1] # all columns except the last one (rating)
    labels = df.iloc[:, -1] # last column (rating)

    selector = SelectPercentile(percentile=10)
    x = selector.fit_transform(features, labels)
    selected_features = selector.get_feature_names_out()

    x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.2, random_state=1)

    xgb_bonus_clf = xgb.XGBClassifier(max_depth=4, seed=2)
    xgb_bonus_clf.fit(x_train, y_train)

    return xgb_bonus_clf, selected_features

In [96]:
def bonus_ass_rating_prediction(table_path, trained_model, select_features):
    """
        The function Receives the path to one of the foloowing tables: 'reviews_rollout.csv'
        or 'reviews_training.csv' and returns the predicted values in dataframe along with the
        respected ids.

        Assumptions:
        1. Both tables contain an id column
        2. Rating Column does not appear in any table
        3. model was Trained on the same tryp of population
    """    
    df = pd.read_csv(table_path)
    selected = df[select_features]
    y_pred = trained_model.predict(selected)

    return pd.concat([df["ID"], pd.Series(y_pred, name='Rating')], axis=1)

In [103]:
trained_model, selected_features = train_bonus_ass_model()
y_pred_rev_training =   bonus_ass_rating_prediction("Documents/reviews_training.csv", trained_model, selected_features)
y_pred_rev_rollout =    bonus_ass_rating_prediction("Documents/reviews_rollout.csv", trained_model, selected_features)
y_pred_rev_training

c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1208 1527] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


,ID,Rating
0,2,0
1,14,1
2,18,0
3,93,1
4,95,0
...,...,...
1856,29938,0
1857,29948,0
1858,29969,1
1859,29980,1


##### Merge the predicted values with the new tables:
1. ffp_rollout_X
2. ffp_train

In [106]:
df_reviews_train = pd.read_csv("Documents/ffp_train.csv")
merged_ffp_train = pd.merge(left=df_reviews_train, right=y_pred_rev_training, on=["ID", "ID"])

df_reviews_rollout = pd.read_csv("Documents/ffp_rollout_X.csv")
merged_ffp_rollout = pd.merge(left=df_reviews_rollout, right=y_pred_rev_rollout, on=["ID", "ID"])

,ID,CUSTOMER_GRADE,STATUS_PANTINUM,STATUS_GOLD,STATUS_SILVER,NUM_DEAL,LAST_DEAL,ADVANCE_PURCHASE,FARE_L_Y1,FARE_L_Y2,...,POINTS_L_Y2,POINTS_L_Y3,POINTS_L_Y4,POINTS_L_Y5,COUPON_FLAG,CANCEL_FLAG,CREDIT_FLAG,RELATED_FLAG,BUYER_FLAG,Rating
0,30012,13.531186,0,0,1,4,50.7,29,202.0,194.0,...,209.7,222.9,197.7,187.3,0,0,0,0,0,0
1,30021,8.395871,0,0,0,6,18.7,20,101.3,101.3,...,90.7,104.3,148.2,131.4,0,1,0,0,0,0
2,30043,9.290667,0,0,0,4,26.7,19,87.3,98.0,...,101.7,85.8,80.2,91.7,0,0,0,0,0,1
3,30053,14.187698,0,0,1,3,40.0,16,115.3,112.7,...,139.3,148.5,142.6,156.3,0,0,0,0,0,0
4,30066,7.970428,0,0,0,4,32.0,21,124.7,126.0,...,122.7,104.7,128.6,106.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134,49888,5.957926,0,0,0,2,48.0,19,100.7,91.3,...,77.0,82.1,121.2,107.7,0,0,0,0,0,0
1135,49916,6.983812,0,0,0,1,106.7,23,106.0,110.7,...,142.6,134.9,134.9,106.7,0,0,0,0,0,0
1136,49938,17.362321,0,0,0,1,96.0,16,98.0,94.7,...,65.3,69.7,72.3,68.5,0,0,0,0,0,0
1137,49947,14.064782,0,0,1,1,154.7,20,146.0,152.7,...,126.3,132.9,145.1,139.5,1,0,0,1,0,0


NOTICE: The currenly merged table, throws away values that are not found in both tables, this should be fixed

##### Lets use the model trained on the Bonus assignment data, to create labels for reviews training.